In [1]:
import numpy as np
import pandas as pd
import pylab as plt
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import sys
import pylab as plt
import pandas as pd

In [2]:
class RandomSinewaveData(Dataset):

    def __init__(self, split, seq_len = 128, T = 500, beta = 0.01):
        assert split in {'train', 'test'}
        self.split = split
        self.seq_len = seq_len
        self.beta = beta
        self.T = T
        
    def __len__(self):
        return 10000 # ...

        
    def alphaT(self, t): 
    	alpha = 1-self.beta
    	alphaT = alpha**(t)
    	return alphaT
    
    def forward_noise(self, x, t):
    	alpha_t = self.alphaT(t)
    	# Add Gaussian noise with mean 0 and covariance (1-alpha_t)*I to sqrt(alpha_t) * x
    	noise = torch.randn(size = x.size()) * np.sqrt(1 - alpha_t)
    	return (np.sqrt(alpha_t) * x) + noise
    
    def __getitem__(self, idx):
        x = torch.arange(0, 7, 7/self.seq_len)
        random_phase = np.random.random() * 2 * np.pi 
        x = torch.sin(x + random_phase)
        
        # Sample number of noise steps t to apply between 1 and T
        t = int(np.random.random() * self.T)
        
        # Add forward noising
        y = self.forward_noise(x, t)

        # compute the noise
        noise = y - x

        # return the noisy data and the noise in it
        return y, noise, t

In [3]:
train_dataset = RandomSinewaveData('train', 256)
test_dataset = RandomSinewaveData('test', 256)

In [4]:
train_loader = DataLoader(
            train_dataset,
            shuffle=True,
            pin_memory=True,
            batch_size=100,
            num_workers=0,
            drop_last=True
        )

In [5]:
for batch in train_loader:
    pass

batch = [t.to('cpu') for t in batch]
x, y, t = batch

In [6]:
x.size(), y.size(), t.size()

(torch.Size([100, 256]), torch.Size([100, 256]), torch.Size([100]))

In [7]:
from model import DDPM
model = DDPM(256, 0.01)

In [8]:
from trainer import Trainer
train_config = Trainer.get_default_config()
train_config.learning_rate = 3e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 20000
train_config.num_workers = 0
train_config.batch_size = 100
train_config.eval_iters = 100
train_config.resume = False
train_config.checkpoint_path = '/home/suhas/research/dl/checkpoints/'
trainer = Trainer(train_config, model, train_dataset, test_dataset)

2024-05-08 08:12:36.964316: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 08:12:37.178656: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-08 08:12:37.178682: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-05-08 08:12:38.046047: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

running on device cpu


In [ ]:
trainer.run()

iter_num 100  train_loss: 0.975667417049408 , test_loss:  0.9838844537734985 , last batch loss: 0.9633376598358154
iter_num 200  train_loss: 0.9027020335197449 , test_loss:  0.9024069905281067 , last batch loss: 0.9184257984161377
iter_num 300  train_loss: 0.9022958278656006 , test_loss:  0.9001274704933167 , last batch loss: 0.9484067559242249
iter_num 400  train_loss: 0.8982862830162048 , test_loss:  0.8977109789848328 , last batch loss: 0.9855605363845825
iter_num 500  train_loss: 0.8959603905677795 , test_loss:  0.9011164307594299 , last batch loss: 0.9265532493591309
iter_num 600  train_loss: 0.8935606479644775 , test_loss:  0.8990119695663452 , last batch loss: 0.8663609027862549
iter_num 700  train_loss: 0.9127204418182373 , test_loss:  0.9063230752944946 , last batch loss: 0.8772531747817993
iter_num 800  train_loss: 0.8915033936500549 , test_loss:  0.8976140022277832 , last batch loss: 0.8990994095802307
iter_num 900  train_loss: 0.8918917179107666 , test_loss:  0.892184317111